In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions  import *

from dotenv import load_dotenv
import sys
import os
# Добавляем корневую директорию проекта в sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

from consultant.tools.basic import normalize_path

load_dotenv()

ENVIRONMENT = os.getenv('ENVIRONMENT')
HOME_DIR = os.getenv('HOME_DIR')
FIXTURE_DIR = os.getenv('FIXTURE_DIR')

spark = (SparkSession.builder
        .appName("bank_transactions_etl")
        .getOrCreate())

spark

**READING**

In [9]:
path = normalize_path(*[HOME_DIR, FIXTURE_DIR, 'bank_transactions.csv'])

transactions = spark.read.csv(path, header=True, inferSchema=True)
transactions.printSchema()
transactions.show(100, truncate=False)
transactions.select(count(expr("*"))).show(1)

root
 |-- client_id: integer (nullable = true)
 |-- transaction_id: integer (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- transaction_type: string (nullable = true)
 |-- account_number: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- amount: double (nullable = true)

+---------+--------------+--------------------------+----------------+----------------------+--------+-----------------+
|client_id|transaction_id|transaction_date          |transaction_type|account_number        |currency|amount           |
+---------+--------------+--------------------------+----------------+----------------------+--------+-----------------+
|1        |2614          |2023-10-10 22:09:54.535652|deposit         |GB80CGKK78193385968259|RUB     |446512.86        |
|1        |9447          |2024-01-17 01:21:33.535652|withdrawal      |GB24CDAY42226993544625|RUB     |390925.84        |
|1        |8622          |2023-12-22 01:43:08.535652|deposit         |GB51GJLB98

In [ ]:
correct_amount = transactions \
    .withColumn(
        'amount',
        regexp_replace(
            col('amount'),
            r"(\d+\.\d{2})\d",
            r"\1"
        )
    )

prepared_transactions = correct_amount \
    .dropDuplicates() \
    .na.drop('all')


+--------+
|count(1)|
+--------+
|  754864|
+--------+



In [21]:
spark.stop()